In [1]:
from FunctionSet import *
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri
from outputC import *
import numpy as np

import sympy
from sympy import symbols, IndexedBase, Indexed, Idx, preorder_traversal

Set path where the "ET... .H" files should go.

In [2]:
path = "../Source/WaveEquation/"

Declare indexed variables and names.

In [3]:
Nx, Ny, Nz= symbols('Nx Ny Nz', integer=True)
i = Idx('i', Nx)
j = Idx('j', Ny)
k = Idx('k', Nz)
name = Idx('name')
dx, dy, dz = symbols('dx dy dz')

var_names = ["Phi","Pi"]

phi = IndexedBase('state_fab'+'Phi')[i,j,k,var_names[0]]
pi = IndexedBase('state_fab'+'Pi')[i,j,k,var_names[1]]

Right hand sides in symbolic form.

The variables are also initialized in symbolic form using sympy functions.

In [4]:
RHS_dotPHI = pi
RHS_dotPI = Dc2(phi,'x')+Dc2(phi,'y')

x, y = sp.symbols("x y")
INIT_PHI = sp.sin(2.0*sp.pi*x)
INIT_PI = -2.0*sp.pi*sp.cos(2.0*sp.pi*x)

These are what the RHSs will look like in AMReXcode.

In [5]:
print(AMReXcode(RHS_dotPHI,var_names,True,var_names[0])+"\n")
print(AMReXcode(RHS_dotPI,var_names,True,var_names[1]))

rhs_fab(i, j, k, Idx::Phi) = state_fab(i, j, k, Idx::Pi);

rhs_fab(i, j, k, Idx::Pi) = (state_fab(i, j - 1, k, Idx::Phi) + state_fab(i, j + 1, k, Idx::Phi) - 2*state_fab(i, j, k, Idx::Phi))/(dx[1]*dx[1]) + (state_fab(i - 1, j, k, Idx::Phi) + state_fab(i + 1, j, k, Idx::Phi) - 2*state_fab(i, j, k, Idx::Phi))/(dx[0]*dx[0]);


We now create and populate the required ".H" files to run our simulation.  They are sent to our "path" location.

In [6]:
createSETUP(path+"ET_Integration_Setup.H",var_names,1)

In [7]:
createINIT(path+"ET_Integration_Init_K.H")

addINIT(path+"ET_Integration_Init_K.H",AMReXcode(INIT_PHI,var_names,declare_state=True,statename=var_names[0]))
addINIT(path+"ET_Integration_Init_K.H",AMReXcode(INIT_PI,var_names,declare_state=True,statename=var_names[1]))

finishINIT(path+"ET_Integration_Init_K.H")

In [8]:
createRHS(path+"ET_Integration_Rhs_K.H")

addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_dotPHI,var_names,declare_rhs=True,rhsname=var_names[0]))
addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_dotPI,var_names,declare_rhs=True,rhsname=var_names[1]))

finishRHS(path+"ET_Integration_Rhs_K.H")